In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = r'C:\Users\tosha\PycharmProjects\anomalies\ukpn-smart-meter-consumption-lv-feeder.csv'
df = pd.read_csv(data_path)
df

,dataset_id,dno,dno_alias,secondary_substation_id,lv_feeder_id,substation_geo_location,aggregated_device_count_active,primary_consumption_active_import,secondary_consumption_active_import,total_consumption_active_import,aggregated_device_count_reactive,total_consumption_reactive_import,data_collection_log_timestamp,insert_time,last_modified_time
0,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0857,83.0,"52.1394219872306, -0.44645766957868",28.0,5745.0,0.0,5745.0,34.0,1139.0,2024-01-01T13:00:00+03:00,2024-02-20T15:00:00+03:00,2024-02-20T15:00:00+03:00
1,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0857,84.0,"52.1394219872306, -0.44645766957868",6.0,824.0,0.0,824.0,9.0,52.0,2024-01-22T06:30:00+03:00,2024-02-20T15:00:00+03:00,2024-02-20T15:00:00+03:00
2,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0860,81.0,"52.1435447516938, -0.536284777985823",56.0,4386.0,0.0,4386.0,64.0,800.0,2024-01-14T05:00:00+03:00,2024-02-12T15:00:00+03:00,2024-02-12T15:00:00+03:00
3,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0860,81.0,"52.1435447516938, -0.536284777985823",56.0,16467.0,0.0,16467.0,64.0,1257.0,2024-01-15T23:00:00+03:00,2024-02-12T15:00:00+03:00,2024-02-12T15:00:00+03:00
4,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0860,81.0,"52.1435447516938, -0.536284777985823",57.0,15814.0,0.0,15814.0,65.0,2050.0,2024-01-23T12:30:00+03:00,2024-02-12T15:00:00+03:00,2024-02-12T15:00:00+03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3074320,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0912,6.0,"52.1346909015886, -0.486154645979534",40.0,4023.0,0.0,4023.0,48.0,390.0,2024-01-14T08:30:00+03:00,2024-02-16T15:00:00+03:00,2024-02-16T15:00:00+03:00
3074321,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0913,84.0,"52.1388750756072, -0.505463497795412",12.0,2716.0,0.0,2716.0,12.0,24.0,2024-01-10T13:30:00+03:00,2024-02-14T15:00:00+03:00,2024-02-14T15:00:00+03:00
3074322,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0915,82.0,"52.1266502175944, -0.462223164143835",12.0,7485.0,0.0,7485.0,18.0,90.0,2024-01-08T08:30:00+03:00,2024-02-18T15:00:00+03:00,2024-02-18T15:00:00+03:00
3074323,UKPN_Feb_2024,UK Power Networks,UKPN_EPN,EPN-S0000004U0915,83.0,"52.1266502175944, -0.462223164143835",19.0,3546.0,0.0,3546.0,23.0,847.0,2024-01-13T19:00:00+03:00,2024-02-18T15:00:00+03:00,2024-02-18T15:00:00+03:00


In [3]:
selected_columns = ['secondary_substation_id', 'lv_feeder_id', 'aggregated_device_count_active', 
                    'primary_consumption_active_import', 'data_collection_log_timestamp']
df = df[selected_columns]
df

,secondary_substation_id,lv_feeder_id,aggregated_device_count_active,primary_consumption_active_import,data_collection_log_timestamp
0,EPN-S0000004U0857,83.0,28.0,5745.0,2024-01-01T13:00:00+03:00
1,EPN-S0000004U0857,84.0,6.0,824.0,2024-01-22T06:30:00+03:00
2,EPN-S0000004U0860,81.0,56.0,4386.0,2024-01-14T05:00:00+03:00
3,EPN-S0000004U0860,81.0,56.0,16467.0,2024-01-15T23:00:00+03:00
4,EPN-S0000004U0860,81.0,57.0,15814.0,2024-01-23T12:30:00+03:00
...,...,...,...,...,...
3074320,EPN-S0000004U0912,6.0,40.0,4023.0,2024-01-14T08:30:00+03:00
3074321,EPN-S0000004U0913,84.0,12.0,2716.0,2024-01-10T13:30:00+03:00
3074322,EPN-S0000004U0915,82.0,12.0,7485.0,2024-01-08T08:30:00+03:00
3074323,EPN-S0000004U0915,83.0,19.0,3546.0,2024-01-13T19:00:00+03:00


In [6]:
df.loc[:, 'active_power_normalized'] = (df['primary_consumption_active_import'] / df['aggregated_device_count_active']).round()

In [8]:
rows = []

In [16]:
for substation in df['secondary_substation_id'].unique():
    for lv_feeder in df[df['secondary_substation_id'] == substation]['lv_feeder_id'].unique():
        row = df[(df['secondary_substation_id'] == substation) & (df['lv_feeder_id'] == lv_feeder)]
        row = row.sort_values(by='data_collection_log_timestamp', ascending=True)['active_power_normalized'].to_numpy()
        rows.append(row)

In [17]:
rows = [row for row in rows if len(row) == 1488]

In [18]:
rows = [row for row in rows if np.count_nonzero(np.isnan(row)) == 0]

In [19]:
filtered_rows = pd.DataFrame(data=np.array(rows))
filtered_rows

,0,1,2,3,4,5,6,7,8,9,...,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487
0,181.0,145.0,131.0,132.0,141.0,131.0,109.0,95.0,98.0,93.0,...,296.0,337.0,284.0,232.0,243.0,216.0,219.0,214.0,236.0,180.0
1,282.0,198.0,162.0,300.0,244.0,211.0,167.0,158.0,122.0,86.0,...,326.0,332.0,294.0,364.0,338.0,205.0,290.0,168.0,106.0,91.0
2,225.0,235.0,232.0,153.0,136.0,135.0,132.0,126.0,114.0,119.0,...,293.0,313.0,357.0,276.0,285.0,285.0,264.0,272.0,210.0,160.0
3,134.0,131.0,126.0,126.0,118.0,105.0,108.0,130.0,88.0,82.0,...,328.0,273.0,305.0,258.0,222.0,246.0,200.0,154.0,130.0,106.0
4,434.0,357.0,341.0,463.0,474.0,429.0,354.0,328.0,346.0,339.0,...,358.0,263.0,287.0,289.0,241.0,241.0,234.0,235.0,222.0,157.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,202.0,221.0,369.0,294.0,247.0,200.0,139.0,150.0,101.0,132.0,...,609.0,546.0,438.0,385.0,334.0,239.0,257.0,291.0,160.0,207.0
1638,111.0,136.0,105.0,109.0,84.0,90.0,92.0,88.0,88.0,77.0,...,289.0,302.0,266.0,220.0,266.0,272.0,171.0,140.0,96.0,132.0
1639,180.0,348.0,434.0,392.0,378.0,374.0,352.0,438.0,449.0,444.0,...,316.0,275.0,310.0,277.0,267.0,318.0,294.0,186.0,154.0,101.0
1640,127.0,99.0,104.0,112.0,79.0,127.0,121.0,128.0,125.0,131.0,...,340.0,361.0,295.0,188.0,173.0,188.0,201.0,165.0,110.0,97.0


In [21]:
filtered_rows.to_csv('data.csv', index=False)